## Network definition
Class definition to create a CNN

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features



### Print out the network features

In [39]:
# then print the network
net = Net()
print(net)

params = list(net.parameters())
print(len(params))
print(params[0].size())

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
torch.Size([6, 1, 5, 5])


### Run the forward network

In [40]:
input = torch.randn(1,1,32,32)
target = torch.randn(10)
print(target)
target = target.view(1,-1)
print(target)
output = net(input)
print(output)

tensor([ 1.1089,  0.9632,  2.2223, -0.5319, -1.7364, -1.9147,  2.3541, -0.8722,
        -0.8328,  0.8116])
tensor([[ 1.1089,  0.9632,  2.2223, -0.5319, -1.7364, -1.9147,  2.3541, -0.8722,
         -0.8328,  0.8116]])
tensor([[-0.0186,  0.0869,  0.0664,  0.0808,  0.0520,  0.0057,  0.0032, -0.0037,
          0.1646,  0.0482]], grad_fn=<ThAddmmBackward>)


### Compute the loss function

In [41]:
# set the type of loss we want
criterion  = nn.MSELoss()

# ... then compute the cost
loss = criterion(output,target)
print(loss)
print()

# print the loss gradient function
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])


tensor(2.1807, grad_fn=<MseLossBackward>)



### Run the backward network and compute gradients

In [42]:
# print network parameters again
print(net.fc1.weight)

manually = False 
# Zero out all gradient buffers 
net.zero_grad()
if manually:


    print('conv1.bias.grad before backward')
    print(net.conv1.bias.grad)

    # run the backward prop step
    loss.backward(retain_graph=False)

    print('conv1.bias.grad after backward')
    print(net.conv1.bias.grad)




Parameter containing:
tensor([[ 0.0400,  0.0066, -0.0470,  ...,  0.0185, -0.0099,  0.0482],
        [ 0.0245, -0.0361, -0.0487,  ..., -0.0323,  0.0277, -0.0156],
        [ 0.0143, -0.0211,  0.0251,  ...,  0.0365,  0.0321,  0.0488],
        ...,
        [-0.0129,  0.0295,  0.0476,  ...,  0.0083, -0.0265,  0.0416],
        [-0.0146, -0.0217,  0.0301,  ...,  0.0192, -0.0221,  0.0460],
        [ 0.0082,  0.0436,  0.0106,  ..., -0.0498, -0.0082, -0.0424]],
       requires_grad=True)


### Recompute & Update the weights based on errors

In [43]:
learning_rate = 0.01

# doing it manually here
if manually:
    for f in net.parameters():
        # for each data subtract the product of the grad of the data and the learning rate from the data
        #.sub_ is to subtract
        f.data.sub_(f.grad.data * learning_rate)
    


### Alternatively use inbuilt optimizers to do the gradient descent

In [44]:
if not manually:
    optimizer = optim.SGD(net.parameters(), lr = learning_rate)

    # training loop
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    